In [76]:
%pylab inline

from library import lasagnennet as LN
from library import features as F
from library import scale as S
from library import validate as V
import numpy as np
import theano as TH
import pandas as pa
import lasagne as L

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [77]:
train_feats = pa.read_csv('train_feats.csv')
train_targets = pa.read_csv('train_targets.csv')
val_feats = pa.read_csv('validation_feats.csv')
val_targets = pa.read_csv('validation_targets.csv')

In [78]:
tx = np.asarray(train_feats, dtype=TH.config.floatX)
tyall = np.asarray(train_targets, dtype=TH.config.floatX)
ty = np.asarray(train_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)
vx = np.asarray(val_feats, dtype=TH.config.floatX)
vyall = np.asarray(val_targets, dtype=TH.config.floatX)
vy = np.asarray(val_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)

In [79]:
ty = ty.ravel()
vy = vy.ravel()

In [80]:
from sklearn import svm

In [101]:
clf = svm.SVC(verbose=True, max_iter=100, class_weight={0:1.3, 1:1}, probability=True)
clf.fit(tx, ty)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight={0: 1.3, 1: 1}, coef0=0.0, degree=3,
  gamma=0.0, kernel='rbf', max_iter=100, probability=True,
  random_state=None, shrinking=True, tol=0.001, verbose=True)

In [108]:
probs = clf.predict_proba(vx)

In [127]:
thresh = np.percentile(probs[:,1], 99.99)
pred = np.int32(probs[:,1]>=thresh)
print np.sum(pred)

7


In [128]:
output = vyall.swapaxes(0,1)
output[2] = pred
output = output.swapaxes(0,1)

In [129]:
outfile = open('out.csv', 'w')
outfile.write('ID,TIME,LABEL,ICU\n')
for i in output:
    outfile.write(','.join([str(int(j)) for j in i])+'\n')
outfile.close()

In [131]:
reload(V)
V.validate('out.csv', './Training_Dataset/id_label_train.csv')

tp :  1 , tn :  827 , fp :  1 , fn :  71
sensitivity :  0.0138888888889 , specificity :  0.998792270531
accuracy :  0.92 median prediction time :  4820.5
specificity < 0.99 or sensitivity = 0            or medianPredictionTime < 5
finalScore :  0


(0.013888888888888888, 0.998792270531401, 0.92, 4820.5, 0)